# Yeh concrete data regressions with sklearn

https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength        
http://scikit-learn.org/stable/index.html   
http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html   
http://scikit-learn.org/stable/modules/classes.html  
http://scikit-learn.org/stable/supervised_learning.html 

### imports

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:0.2f}'.format

from bokeh.charts import Histogram, Line, Dot, Bar, show
from bokeh.layouts import row, column
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LabelSet, Arrow, OpenHead, NormalHead, VeeHead, Label
from bokeh.models import LinearColorMapper, palettes
from bokeh.colors import RGB
from bokeh.io import output_notebook
output_notebook()

from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

Loading BokehJS ...

### utilities

In [2]:
def plotSimple(title, xTitle, yTitle, x, y, y_range=(0, 100)):
    p = figure(title = title, plot_width=300, plot_height=250, y_range=y_range)
    p.circle(x, y, size=3, color="red", alpha=1)
    p.line(x, y, color="red", alpha=1)
    p.xaxis.axis_label = xTitle
    p.yaxis.axis_label = yTitle
    return p


def plotExperiment(xName, yName, experiment, df, model, blurcount, y_range=(0, 100)):
    def blur():
        lottery = np.random.random_sample(size=df.count()[0])
        lottery = [0.00 <= r < 0.70 for r in lottery]
        X = df[lottery][df.X]
        y = df[lottery][df.y]
        model.fit(X=X, y=y)
        Y = model.predict(experiment)
        p.line(experiment[xName], Y, color="lightblue",line_width=6, line_alpha=1)    
        p.line(experiment[xName], Y, color="blue",line_width=0, line_alpha=0.5)    
    p = figure(title = type(model).__name__, plot_width=300, plot_height=250, y_range=y_range)
    [blur() for i in range(blurcount)]
    Y = model.predict(X=experiment)
    p.line(experiment[xName], Y, color="red", line_width=3, line_alpha=1)    
    p.xaxis.axis_label = xName
    p.yaxis.axis_label = yName
    return p    
    
def plotRegression(df, model):  
    def plotRegression1(selection, color, alpha=1):
        X = df[selection][df.X]
        y = df[selection][df.y]
        Y = model.predict(X)
        r2 = np.corrcoef(y, Y)[0,1]
        se = np.std(y - Y)
        p.circle(y.tolist(), Y, size=3, color=color, alpha=alpha)
        p.line(y, y, color="red")
        return [r2,se]
    model.fit(X=df[df.train][df.X], y=df[df.train][df.y])    
    p = figure(title = type(model).__name__, plot_width=400, plot_height=400)
    trainr2, trainse = plotRegression1(df.train, "red")
    validr2, validse = plotRegression1(df.valid, "blue")
    testr2,  testse  = plotRegression1(df.test,  "green")
    r2text = "R2: train={r1: 6.2f}, valid={r2: 6.2f}, test={r3: 6.2f}".format(r1=trainr2,r2=validr2,r3=testr2)
    setext = "SE: train={r1: 6.2f}, valid={r2: 6.2f}, test={r3: 6.2f}".format(r1=trainse,r2=validse,r3=testse)
    p.text(x=[0.5], y = [0.2], text=[r2text], text_font_size="10pt")
    p.text(x=[0.5], y = [0.0], text=[setext], text_font_size="10pt")    
    p.xaxis.axis_label = "measured"
    p.yaxis.axis_label = "predicted"
    return p

def plotImportance(importances):
    importances["variable"] = importances.index
    importances.sort_values(by="importance", ascending=False, inplace=True)
    importances.index = [i+1 for  i in range(importances.variable.count())]
    p = Bar(importances, values='importance', title="importances", legend=None, width = 400, height = 400, color="Coral")
    p.y_range.end = p.y_range.end * 1.5
    source = ColumnDataSource(importances)
    labels = LabelSet(x="index", y="importance", text="variable", text_font_size="10pt", text_color="blue", source=source, angle = 45, angle_units="deg")
    p.add_layout(labels)
    return p

def draw_PCAcomponents(df, ca, cb):
    def draw_component(df, feature, ca, cb):
        x_end=df[feature][ca]
        y_end=df[feature][cb]
        rl = (x_end**2+y_end**2)**0.5
        xl = x_end/rl * (1+rl)
        yl = y_end/rl * (1+rl)
        xl = x_end * 2
        yl = y_end * 2
        angle = np.arctan2(y_end,x_end)
        p.add_layout(Arrow(end=VeeHead(size=0), line_color="lightgrey", x_end=xl, y_end=yl))
        p.add_layout(Arrow(end=VeeHead(size=5), line_color="red", x_end=x_end, y_end=y_end))
        p.add_layout(Label(text = feature, x = xl, y = yl, text_font_size="8pt", text_color="blue", angle=angle))
    size = 1.5
    p = figure(title = "title", plot_width=400, plot_height=400, y_range=(-size,size), x_range=(-size,size))
    p.xaxis.axis_label = "C_" + str(ca)
    p.yaxis.axis_label = "C_" + str(cb)
    a = np.array([a*2*np.pi/36 for a in range(37)])
    [p.line(r*np.cos(a), r*np.sin(a), color="grey", alpha=r) for r in [0.2,0.4,0.6,0.8,1.0]]
    [draw_component(df, feature, ca, cb) for feature in df.columns.tolist()]
    return p

def myCorrelationPlot(df, pw=300, ph=300, tt="correlations"):
    colNames = df.columns.tolist()
    rownames = df.index.tolist()
    x = [c for r in rownames for c in colNames]
    y = [r for r in rownames for c in colNames]
    corarr = [df[c][r] for r in rownames for c in colNames]
    colors = [RGB(255*(1-x)/2,255*(1+x)/2,0,0.7) for x in corarr]
    p = figure(title=tt, x_range=colNames, y_range=rownames, plot_width=pw, plot_height=ph, toolbar_location="right")
    p.rect(x, y, color=colors, width=1, height=1)
    p.xaxis.major_label_orientation = 3.14159/2
    c = myColorBar(75, ph)
    return row(p,c)

def myColorBar(pw=75, ph=300, tt="colors"):
    from bokeh.models import CategoricalAxis
    corarr = [-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    colors = [RGB(255*(1-x)/2,255*(1+x)/2,0,0.7) for x in corarr]
    colNames = ["color"]
    rownames = [str(x) for x in corarr]
    x = [c for r in rownames for c in colNames]
    y = [r for r in rownames for c in colNames]
    p = figure(title=tt, x_range=colNames, y_range=rownames, plot_width=pw, plot_height=ph, toolbar_location=None)
    p.rect(x, y, color=colors, width=1, height=1)
    p.xaxis.major_label_orientation = 3.14159/2
    return p

## descriptive statistics

### reading the concrete data from  I-Cheng Yeh (1998)
http://www.sciencedirect.com/science/article/pii/S0008884698001653   
https://www.researchgate.net/publication/222447231_Modeling_of_Strength_of_High-Performance_Concrete_Using_Artificial_Neural_Networks_Cement_and_Concrete_research_2812_1797-1808
https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

In [3]:
# url = "Concrete_Data_Yeh.csv"
url = "https://raw.githubusercontent.com/maajdl/Yeh-concrete-data-sklearn/master/Concrete_Data_Yeh.csv"
beton = pd.read_csv("Concrete_Data_Yeh.csv")
beton['w/c'] = beton['water']/beton['cement']

In [4]:
beton.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa,w/c
0,540.00,0.00,0.00,162.00,2.50,1040.00,676.00,28,79.99,0.30
1,540.00,0.00,0.00,162.00,2.50,1055.00,676.00,28,61.89,0.30
2,332.50,142.50,0.00,228.00,0.00,932.00,594.00,270,40.27,0.69
3,332.50,142.50,0.00,228.00,0.00,932.00,594.00,365,41.05,0.69
4,198.60,132.40,0.00,192.00,0.00,978.40,825.50,360,44.30,0.97


### count, mean, std, ...

In [5]:
display(beton.describe())

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa,w/c
count,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00
mean,281.17,73.90,54.19,181.57,6.20,972.92,773.58,45.66,35.82,0.75
std,104.51,86.28,64.00,21.35,5.97,77.75,80.18,63.17,16.71,0.31
min,102.00,0.00,0.00,121.80,0.00,801.00,594.00,1.00,2.33,0.27
25%,192.38,0.00,0.00,164.90,0.00,932.00,730.95,7.00,23.71,0.53
50%,272.90,22.00,0.00,185.00,6.40,968.00,779.50,28.00,34.45,0.68
75%,350.00,142.95,118.30,192.00,10.20,1029.40,824.00,56.00,46.14,0.94
max,540.00,359.40,200.10,247.00,32.20,1145.00,992.60,365.00,82.60,1.88


### histograms
http://bokeh.pydata.org/en/0.12.5/docs/reference/charts.html#histogram

In [6]:
hist1 = Histogram(beton['csMPa'],                   title="Compressive strength", bins=25, width = 240, height = 240)
hist2 = Histogram(beton['cement'],                  title="cement",               bins=25, width = 240, height = 240)
hist3 = Histogram(beton['water'],                   title="water",                bins=25, width = 240, height = 240)
hist4 = Histogram(beton['w/c'],                     title="w/c",                  bins=25, width = 240, height = 240)
show(row(hist1,hist2,hist3,hist4))

### correlations

In [7]:
p = myCorrelationPlot(beton.corr())
show(p)

### PCA
http://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_vs_lda.html#sphx-glr-auto-examples-decomposition-plot-pca-vs-lda-py

In [8]:
from sklearn.decomposition import PCA
betonNorm2 = (beton-beton.mean())/beton.std()
pca = PCA().fit(betonNorm2)

var = pca.explained_variance_ratio_
varcum = [sum([w for w in var if w>=v]) for v in var]

p = plotSimple("cumulated variance","component","cum. var.", range(9), varcum, (0.2,1.1))
show(p)

np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
pd.options.display.float_format = '{:0.2f}'.format

pcadf = pd.DataFrame(pca.components_)
pcadf = pcadf.T
pcadf.columns = beton.columns

p1 = draw_PCAcomponents(pcadf, 0, 1)
p2 = draw_PCAcomponents(pcadf, 1, 2)
p3 = draw_PCAcomponents(pcadf, 2, 3)
p4 = draw_PCAcomponents(pcadf, 3, 4)
show(column(row(p1,p2),row(p3,p4)))

## regressions, machine learning

### normalised data and variables

In [9]:
betonNorm = beton/beton.mean()
betonNorm.X = ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age']
betonNorm.y = 'csMPa'

### define training set, validation set, test set
random splitting  implies that different runs may yield different results, allowing futher checks

In [10]:
lottery = np.random.random_sample(size=betonNorm.cement.count())
betonNorm.train = [0.00 <= r < 0.70 for r in lottery]
betonNorm.valid = [0.70 <= r < 0.85 for r in lottery]
betonNorm.test  = [0.85 <= r < 1.00 for r in lottery]

### LinearRegression
http://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares

In [11]:
from sklearn import linear_model
linear = linear_model.LinearRegression()
p1 = plotRegression(betonNorm, linear)

importances = abs(linear.coef_)/np.linalg.norm(linear.coef_)
importances = pd.DataFrame(data=importances, columns=['importance'], index = betonNorm.X)
p2 = plotImportance(importances)

show(row(p1,p2))

### RANSAC Regressor (RANdom SAmple Consensus)
http://scikit-learn.org/stable/modules/linear_model.html#ransac-random-sample-consensus

#### Regression

In [12]:
from sklearn import linear_model
ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
p1 = plotRegression(betonNorm, ransac)

inliers    = betonNorm[betonNorm.train]
inliers.y  = inliers[betonNorm.y]
inliers.X  = inliers[betonNorm.X]
outliers   = betonNorm[betonNorm.train][np.logical_not(ransac.inlier_mask_)]
outliers.y = outliers[betonNorm.y]
outliers.X = outliers[betonNorm.X]
outliers.p = ransac.predict(X=outliers.X)

from sklearn import linear_model
ransacInliers = linear_model.LinearRegression()
p2 = plotRegression(betonNorm, ransacInliers)
p2.circle(outliers.y, outliers.p, size=5, color="red", alpha=0.1)

importances = abs(ransac.estimator_.coef_)/np.linalg.norm(ransac.estimator_.coef_)
importances = pd.DataFrame(data=importances, columns=['importance'], index = betonNorm.X)
p3 = plotImportance(importances)

show(row(p2,p3))

#### Inliers   and   Outliers 
Observe: outliers mainly have a larger mean age

In [13]:
means = pd.DataFrame(inliers.mean(), columns=["inliers mean"])
means["outliers mean"] = outliers.mean()
display(means.T)

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa,w/c
inliers mean,0.99,1.00,1.03,1.00,1.02,1.00,1.00,1.02,0.99,1.01
outliers mean,1.04,1.31,0.72,1.01,0.75,1.00,0.97,1.53,0.96,0.97


### DecisionTreeRegressor
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

todo:  try to display the tree, or find an alternative package

In [14]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(min_samples_split=10)
p1 = plotRegression(betonNorm, tree)

importances = pd.DataFrame(data=tree.feature_importances_, columns=['importance'], index = betonNorm.X)
p2 = plotImportance(importances)

show(row(p1,p2))

### RandomForestRegressor
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [15]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=10, max_depth=None, min_samples_split=10, random_state=0)
p1 = plotRegression(betonNorm, forest)

importances = pd.DataFrame(data=forest.feature_importances_, columns=['importance'], index = betonNorm.X)
p2 = plotImportance(importances)

show(row(p1,p2))

### Support Vector Machine Regression
http://scikit-learn.org/stable/modules/svm.html#svm-regression

In [16]:
from sklearn import svm
svr = svm.SVR(kernel='rbf', C=1)
p1 = plotRegression(betonNorm, svr)

show(row(p1))

### neural_network MLPRegressor
http://scikit-learn.org/stable/modules/neural_networks_supervised.html#regression
http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

In [17]:
from sklearn.neural_network import MLPRegressor
neural = MLPRegressor(solver='lbfgs', alpha=0.01, hidden_layer_sizes=(8), activation="logistic", max_iter=10000)
p1 = plotRegression(betonNorm, neural)  

importances = [max(abs(neural.coefs_[0][i])) for i in range(8)]
importances = pd.DataFrame(data=importances, columns=['importance'], index = betonNorm.X)
p2 = plotImportance(importances)

show(row(p1,p2))

### W/C effect  on 28d strengths
in red:  predictions based on the training set   
in blue: predictions from random training sets

In [18]:
nsteps                = 100
experiment1                  = beton.head(nsteps).copy()
experiment1.cement           = beton.cement.mean()
experiment1.slag             = beton.slag.mean()
experiment1.flyash           = beton.flyash.mean()
experiment1.water            = beton.water.mean()
experiment1.superplasticizer = beton.superplasticizer.mean()
experiment1.coarseaggregate  = beton.coarseaggregate.mean()
experiment1.fineaggregate    = beton.fineaggregate.mean()
experiment1.age              = beton.age.mean()
experiment1.age              = 28
xxx                          = np.linspace(-50,50,nsteps)
experiment1.water            = experiment1.water  + xxx
experiment1.cement           = experiment1.cement - 3.15 * xxx
WCratio                      = experiment1.water / experiment1.cement
experiment1                  = experiment1/beton.mean()
experiment1                  = experiment1[betonNorm.X]

p1 = plotExperiment("water", "csMPa", experiment1, betonNorm, linear, 20, y_range=(0, 3))
p2 = plotExperiment("water", "csMPa", experiment1, betonNorm, ransac, 20, y_range=(0, 3))
p3 = plotExperiment("water", "csMPa", experiment1, betonNorm, tree, 20, y_range=(0, 3))
p4 = plotExperiment("water", "csMPa", experiment1, betonNorm, forest, 20, y_range=(0, 3))
p5 = plotExperiment("water", "csMPa", experiment1, betonNorm, svr, 20, y_range=(0, 3))
p6 = plotExperiment("water", "csMPa", experiment1, betonNorm, neural, 20, y_range=(0, 3))

show(column(row(p1, p2, p3), row(p4, p5, p6)))

### age  effect  on strengths
in red:  predictions based on the training set   
in blue: predictions from random training sets

In [19]:
nsteps                = 50
experiment2                  = beton.head(nsteps).copy()
experiment2.cement           = beton.cement.mean()
experiment2.slag             = beton.slag.mean()
experiment2.flyash           = beton.flyash.mean()
experiment2.water            = beton.water.mean()
experiment2.superplasticizer = beton.superplasticizer.mean()
experiment2.coarseaggregate  = beton.coarseaggregate.mean()
experiment2.fineaggregate    = beton.fineaggregate.mean()
experiment2.age              = np.linspace(1,100,nsteps)
experiment2                  = experiment2/beton.mean()
experiment2                  = experiment2[betonNorm.X]

p1 = plotExperiment("age", "csMPa", experiment2, betonNorm, linear, 20, y_range=(0, 3))
p2 = plotExperiment("age", "csMPa", experiment2, betonNorm, ransac, 20, y_range=(0, 3))
p3 = plotExperiment("age", "csMPa", experiment2, betonNorm, tree, 20, y_range=(0, 3))
p4 = plotExperiment("age", "csMPa", experiment2, betonNorm, forest, 20, y_range=(0, 3))
p5 = plotExperiment("age", "csMPa", experiment2, betonNorm, svr, 20, y_range=(0, 3))
p6 = plotExperiment("age", "csMPa", experiment2, betonNorm, neural, 20, y_range=(0, 3))

show(column(row(p1, p2, p3), row(p4, p5, p6)))

In [20]:
from bokeh.models.widgets import Panel, Tabs
t1 = Panel(child=p1, title=p1.title.text)
t2 = Panel(child=p2, title=p2.title.text)
t3 = Panel(child=p3, title=p3.title.text)
t4 = Panel(child=p4, title=p4.title.text)
t5 = Panel(child=p5, title=p5.title.text)
t6 = Panel(child=p6, title=p6.title.text)
layout = Tabs(tabs=[t1,t2,t3,t4,t5,t6])

In [21]:
show(layout)